In [3]:
import numpy as np # linear algebra
import pandas as pd # data processing train, test, sample

import os
print(os.listdir("input_ensemble"))

['sample_submission.csv', 'test.csv', 'train.csv']


In [13]:
# Inport tensorflow lib
import tensorflow as tf
import keras

In [16]:
# Import library keras backend dengan perubahan nama K
from keras import backend as K
K.set_image_data_format('channels_first')

In [18]:
# Proses train data
train = pd.read_csv('input_ensemble/train.csv')
print (train.shape)
train.head()

(42000, 785)


,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [19]:
# melakukan testing pada hasil training
test = pd.read_csv('input_ensemble/test.csv')
print(test.shape)
test.head()

(28000, 784)


,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [20]:
# memisahkan label dari data training dan membuat x_label
y_train = train['label']
x_train = train.drop(labels=['label'],axis=1)
x_test = test


In [21]:
# menentukan nilai dari data
x_train = x_train.values.astype('float32')
y_train = y_train.values.astype('int32')
x_test = test.values.astype('float32')

In [22]:
# penetapan random seed untuk di olah kembali
random_seed = 7
np.random.seed(random_seed)

**
Converting Output into one hot code**

A one hot encoding is a representation of categorical variables as binary vectors. This first requires that the categorical values be mapped to integer values. Then, each integer value is represented as a binary vector that is all zero values except the index of the integer, which is marked with a 1 and as this is a multi classification problem so we can convert the output class values into one-hot format which is simply a binary matrix, i.e.

value 0 will be converted to one-hot format as [1, 0, 0, 0, 0, 0, 0, 0, 0]

value 1 will be converted to one-hot format as [0, 1, 0, 0, 0, 0, 0, 0, 0] etc

In [23]:
# input encode
y_train = np_utils.to_categorical(y_train)
num_classes = y_train.shape[1]
num_classes

10

In [24]:
# Melakukan shaping pada gambar untuk cnn 2 dimensi input
x_train = x_train.reshape(x_train.shape[0], 1, 28, 28).astype('float32')
x_test = x_test.reshape(x_test.shape[0], 1, 28, 28).astype('float32')
num_pixels = x_train.shape[1]
print (num_pixels, x_train.shape, x_test.shape)

1 (42000, 1, 28, 28) (28000, 1, 28, 28)


In [32]:
nn = 10
model = [0]*nn

for j in range(nn):
    model[j] = Sequential()
    model[j].add(keras.layers.Conv2D(filters=32, kernel_size=(3, 3), input_shape=(1, 28, 28), activation='relu',data_format='channels_first'))
    model[j].add(keras.layers.BatchNormalization())
    model[j].add(keras.layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu'))
    model[j].add(keras.layers.BatchNormalization())
    model[j].add(keras.layers.Conv2D(32, kernel_size = 5, strides=2, padding='same', activation='relu'))
    model[j].add(keras.layers.BatchNormalization())
    model[j].add(keras.layers.Dropout(0.4))
    
    model[j].add(keras.layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
    model[j].add(keras.layers.BatchNormalization())
    model[j].add(keras.layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
    model[j].add(keras.layers.BatchNormalization())
    model[j].add(keras.layers.Conv2D(64, kernel_size = 5, strides=2, padding='same', activation='relu'))
    model[j].add(keras.layers.BatchNormalization())
    model[j].add(keras.layers.Dropout(0.4))
    
    model[j].add(keras.layers.Conv2D(128, kernel_size = 4, activation='relu'))
    model[j].add(keras.layers.BatchNormalization())
    model[j].add(keras.layers.Flatten())
    model[j].add(keras.layers.Dropout(0.4))
    model[j].add(keras.layers.Dense(10, activation='softmax'))
    
    # kompilasi pada model
    model[j].compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [34]:
# doing data augmentation to decrease overfitting
datagen = keras.preprocessing.image.ImageDataGenerator(
        rotation_range=10,
        zoom_range = 0.1, 
        width_shift_range=0.1,
        height_shift_range=0.1,
        vertical_flip=False)


datagen.fit(x_train)

In [ ]:
# mengurangi learning rate epoch
annealer = keras.callbacks.LearningRateScheduler(lambda x: 1e-3 * 0.95 ** x)
history = [0] * nn
epochs = 50

# proses fit ada model (error)
for j in range(nn):
    x_train2, x_val, y_train2, y_val = train_test_split(x_train, y_train, test_size = 0.10, random_state=random_seed)
    history[j] = model[j].fit_generator(datagen.flow(x_train2,y_train2, batch_size=64),
                              epochs = epochs, validation_data = (x_val,y_val),
                              verbose = 0, steps_per_epoch=(len(x_train)//64),validation_steps=(len(x_val)//64),callbacks=[annealer])
    print("CNN {0:d}: Epochs={1:d}, Train accuracy={2:.5f}, Validation accuracy={3:.5f}".format(
        j+1,epochs,max(history[j].history['acc']),max(history[j].history['val_acc']) ))

In [ ]:
# Present result from ensemble process
results = np.zeros( (x_test.shape[0],10) )
for j in range(nn):
    results = results + model[j].predict(x_test)
results = np.argmax(results,axis = 1)
results = pd.Series(results,name="Label")
submission = pd.concat([pd.Series(range(1,28001),name = "ImageId"),results],axis = 1)
submission.to_csv("ENSEMBLE.csv",index=False)